In [1]:
from hybrik.datasets import MixDataset, PW3D, H36MDataset

/home/aiteam/miniconda3/envs/hy-data2/lib/python3.7/site-packages/tqdm-4.64.0-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_file = '/home/aiteam/tykim/HybrIK/configs/256x192_adam_lr1e-3-res34_smpl_3d_base_2x_single_nc_test.yaml'

In [3]:
import yaml
from easydict import EasyDict as edict

with open(config_file) as f:
  cfg = edict(yaml.load(f, Loader=yaml.FullLoader))

In [4]:
train_dataset = H36MDataset(cfg=cfg,train=True)

In [5]:
import torch
import numpy as np
import random
def _init_fn(worker_id):
    np.random.seed(33)
    random.seed(33)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=True, num_workers=4, worker_init_fn=_init_fn, pin_memory=False)

In [6]:
import cProfile
cProfile.run('H36MDataset(cfg=config, train=True)')

         31531067 function calls (30906688 primitive calls) in 117.729 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1248752    0.693    0.000    3.641    0.000 <__array_function__ internals>:2(concatenate)
   624376    0.304    0.000    1.067    0.000 <__array_function__ internals>:2(copyto)
   624376    0.322    0.000    6.387    0.000 <__array_function__ internals>:2(stack)
        1    2.881    2.881  117.729  117.729 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:145(_DType_reduce)
        1    0.042    0.042   38.004   38.004 __init__.py:274(load)
        1    0.000    0.000   35.327   35.327 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
   624376    0.186    0.000    1.282    0.000 _methods.py:60(_all)
   312188    3.702    0.000    3.766    0.000 bbox.py:108(bbox_clip_xyxy)
   312188    1.491    0.000    1.583    0.000

In [6]:
import torch.profiler

In [8]:
with torch.profiler.profile(
      activities=[
          torch.profiler.ProfilerActivity.CPU,
          torch.profiler.ProfilerActivity.CUDA],
      schedule=torch.profiler.schedule(
          wait=1,
          warmup=1,
          active=2),
      on_trace_ready=torch.profiler.tensorboard_trace_handler('./result', worker_name='worker0'),
      record_shapes=True,
      profile_memory=True,  # This will take 1 to 2 minutes. Setting it to False could greatly speedup.
      with_stack=True
  ) as p:
    for step, data in enumerate(train_loader, 0):
        print("step:{}".format(step))
        # inputs, labels = data[0].to(device="cuda"), data[1].to(device="cuda")
        if step + 1 >= 4:
            break
        p.step()

step:0
step:1
step:2
step:3
